In [1]:
import os
import openai
import sys
import json
import re
from IPython.display import display, HTML
sys.path.append('../..')

In [2]:
openai.api_key = os.environ['OPENAI_API_KEY']

## LangChain Imports

In [3]:
from langchain_core.output_parsers import StrOutputParser

# https://python.langchain.com/docs/modules/memory/adding_memory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

## Setup and Initialize Chain

In [17]:
system_v1_template = """You are an intelligent Cognitive UI agent that is having a conversation with an user.
Your task is to have a conversation and respond appropriately to the user. You have the ability to generate UI components 
in the form of HTML code and HTML is the output format that the user prefers when needed.
Before you output any HTML, you need to confirm that the HTML is valid, stylized with CSS or other frontend frameworks, attached with 
functional JS scripts and most importantly, it must satisfy the user's requirements. 
Valid HTML must follow the standard HTML file format containing the keyword 'DOCTYPE'.

If the user provides feedback to the generated HTML, you will incorporate their feedback
and make new attempts until they're satisfied. In order to generate HTML, you may need content from the user and it is okay to 
collect information from the user based on the type of UI component they need. 

Once you have enough information from the user, you need to output HTML code; you should
also do this if the user is requesting for HTML. However, if you need more information, you should continue asking for information.
If you have already generated HTML and the user provides feedback and suggest changes, please respond with the updated HTML.

DO NOT tell the user that you will show them the UI component/code without actually including it in your response.
"""

In [18]:
def make_chat(system_template, verbose_mode=False):
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = 0)
    mem = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=mem,
    )
    return chat_llm_chain

In [19]:
def make_chat_summary_mem(system_template, verbose_mode=False):
    '''
    creates a chat model that summarizes the current chat history to work with finite memory.
    '''
    
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = 0)
    # mem = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    mem = ConversationSummaryMemory(memory_key="chat_history", llm=ChatOpenAI(temperature = 0), return_messages=True)
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=mem,
    )
    return chat_llm_chain

In [20]:
html_pattern = re.compile(r'<!DOCTYPE html>.*?</html>', re.DOTALL)
def interact(model, input):
    o = model.predict(human_input=input)
    html_match = html_pattern.search(o)
    s = False
    if html_match:
        s = True
        display(HTML(html_match.group(0)))
    if s:
        o = re.sub(r"```html.*?```", "", o, flags=re.DOTALL)
    return o

## Testing "Unbounded" Memory

In [21]:
model = make_chat(system_v1_template, verbose_mode=False)
interact(model, "Hi")

'Hello! How can I assist you today?'

In [22]:
interact(model, 
"""I want a form for my website""")

'Great! I can help you with that. Could you please provide me with some details about the form you would like to have on your website? For example, what fields should the form include and any specific styling preferences you have in mind?'

In [ ]:
interact(sum_model, """The form needs to be resizable and the submit button should take up the entire space on its row""")

In [ ]:
interact(sum_model, 